In [ ]:
from pymongo import MongoClient

# Source: old DB/collection
client = MongoClient("mongodb://192.168.247.185:27017/")
src_db = client["test2"]
src_collection = src_db["full_dataset"]

destination_db = "test2"
destination_collection = "full_dataset_updated"

# Destination: new DB/collection
dst_db = client[destination_db]
dst_collection = dst_db[destination_collection]

# Clean destination before writing (optional: only if you want a fresh start)
dst_collection.delete_many({})

# Read all documents
documents = list(src_collection.find())
num_samples = len(documents)

for sample_idx, doc in enumerate(documents):
    new_doc = {}

    # Keep original index
    new_doc["index"] = doc.get("index", sample_idx + 1)

    # === PRIMARY ===
    primary = {}
    piston_data = doc.get("piston_values", {}).get("primary_mirror_segments", {})
    for i in range(1, 7):
        seg_key = f"segment_{i}"
        piston = piston_data.get(seg_key, {}).get("piston", 0)
        primary[str(i)] = {
            "piston": piston,
            "tip": 0,
            "tilt": 0
        }
    new_doc["primary"] = primary

    # === SECONDARY ===
    secondary_mirror = doc.get("secondary_mirror", {})
    new_doc["secondary"] = {
        "piston": secondary_mirror.get("piston_nominal", 0),
        "tip": secondary_mirror.get("tip", 0),
        "tilt": secondary_mirror.get("tilt", 0),
        "global_defocus": secondary_mirror.get("piston_defocus", 0)
    }

    # === defocused_nominal_array (optional) ===
    if "defocused_nominal_array" in doc:
        new_doc["defocused_nominal_array"] = doc["defocused_nominal_array"]

    # === psf_nominal_array from image_matrix ===
    if "image_matrix" in doc:
        new_doc["psf_nominal_array"] = doc["image_matrix"]

    # Insert into new collection
    dst_collection.insert_one(new_doc)

    print(f"\rCopied and transformed document {sample_idx + 1}/{num_samples}", end='', flush=True)

print(f"\nDone. All {num_samples} documents have been copied to {destination_collection} in {destination_db}.")


In [1]:
import torch

# Load old state_dict
old_state = torch.load("best_model.pth", map_location="cpu")
new_state = {}

# Updated mapping logic
for key, value in old_state.items():
    if key.startswith("conv1."):
        new_key = "conv_layers.0." + key[len("conv1."):]
    elif key.startswith("conv2."):
        new_key = "conv_layers.1." + key[len("conv2."):]
    elif key.startswith("conv3."):
        new_key = "conv_layers.2." + key[len("conv3."):]
    elif key.startswith("conv4."):
        new_key = "conv_layers.3." + key[len("conv4."):]
    elif key.startswith("conv5."):
        # Old conv5 was single Conv2d, now wrapped in Sequential as conv_layers.4.0
        new_key = "conv_layers.4.0" + key[len("conv5"):]
    elif key.startswith("skip1.0."):
        new_key = "skip_ops.0->2.0." + key[len("skip1.0."):]
    elif key.startswith("skip1.1."):
        new_key = "skip_ops.0->2.1." + key[len("skip1.1."):]
    elif key.startswith("fc1."):
        new_key = "fc_layers.0." + key[len("fc1."):]
    elif key.startswith("fc2."):
        new_key = "fc_layers.1." + key[len("fc2."):]
    elif key.startswith("fc_out."):
        new_key = "fc_out." + key[len("fc_out."):]
    else:
        print("⚠️ Unmapped key:", key)
        new_key = key  # fallback, might be wrong

    new_state[new_key] = value

torch.save(new_state, "resnet.pth")
print("✅ Remapping complete and saved to 'resnet.pth'")


✅ Remapping complete and saved to 'resnet.pth'
